In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import time
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *

In [2]:
def theta_better(theta_1, theta_2):
    if additivity(theta_1) != additivity(theta_2):
        return 1 if additivity(theta_1) < additivity(theta_2) else -1
    else:
        if len(theta_1) != len(theta_2):
            return 1 if len(theta_1) < len(theta_2) else -1
        else:
            return 0
    
def theta_better_opt(theta_1, theta_2):
    if additivity(theta_1) != additivity(theta_2):
        return 1 if additivity(theta_1) < additivity(theta_2) else -1
    else:
        if len(theta_1) != len(theta_2):
            return 1 if len(theta_1) < len(theta_2) else -1
        else:
            if sum([len(i) for i in theta_1]) != sum([len(i) for i in theta_2]):
                return 1 if sum([len(i) for i in theta_1]) < sum([len(i) for i in theta_2]) else -1
            return 0
        
        
def additivity(theta):
    return max([len(i) for i in theta])

In [3]:
def get_candidate_iterator(c, bound = np.inf):
    a = list(set([x[0] for x in c if len(x[0]) > 0] + [x[1] for x in c if len(x[1]) > 0]))
    a = sorted(a, key = lambda x:len(x))
    iterators = {}
    for x in a:
        for k in range(1,min(len(x)+1, bound)):
            iterators[k] = iterators.get(k,[]) + [itertools.combinations(x, k)]
    for k in iterators:
        random.shuffle(iterators[k])
    return iterators


def dfs_thetas_full(preferences, theta, theta_mins, banned = None, bann_opt = False):
    if not banned:
        banned = []
    theta = sorted(theta)
    c = get_connivent(theta, preferences)
    #print("Theta = ", theta, end = "\t")
    #print("c = ", c, end = "\t")
    #print("Banned = ", banned)
    
    if c == None:
        #print("Found ", theta)
        theta_mins.append(theta)
        return 
    
    its = get_candidate_iterator(c)
    for k in its:
        for candidates in its[k]:
            for c in candidates:
                if c in theta:
                    continue
                n_theta = theta + [c]
                dfs_thetas_full(preferences, n_theta, theta_mins)
    return True 

def dfs_thetas_r(preferences, theta, theta_mins = [], banned = None, bann_opt = True):
    if not banned:
        banned = []
    theta = sorted(theta)
    c = get_connivent(theta, preferences)
    print("=====")
    print("Theta = ", theta, end = "\t")
    print("c = ", c, end = "\t")
    print("Banned = ", banned)
    
    if c == None:
        kernels = [theta]
        #kernels += get_kernels(prf, theta)
        for theta in kernels:
            if theta in theta_mins:
                continue
            if len(theta_mins) == 0:
                theta_mins.append(theta)
                return additivity(theta_mins[0]), len(theta_mins[0])
            if theta_better(theta, theta_mins[0]) == 1:
                theta_mins.clear()
                theta_mins.append(theta)
            elif theta_better(theta, theta_mins[0]) == 0:
                theta_mins.append(theta)
        return 
                
    its = get_candidate_iterator(c)
    b = list(banned)
    for k in its:
        for candidates in its[k]:
            for c_i in candidates:
                if len(theta_mins) > 0 and k > additivity(theta_mins[0]):
                    #print("Cutting over", k, " in:", theta , end = " ")
                    break
                if c_i in theta or c_i in b:
                    continue
                n_theta = theta + [c_i]
                if len(theta_mins) > 0 and theta_better(n_theta, theta_mins[0]) == -1:
                    #print("Cutting", c , " in:", theta, end = " ")
                    continue
                #print("Trying candidate ", c)
                dfs_thetas_r(preferences, n_theta, theta_mins, banned = b)
                if bann_opt:
                    b.append(c_i)
    return True

def dfs_thetas_opt(preferences, theta, theta_mins = [], banned = None, bann_opt = False):
    if not banned:
        banned = []
    theta = sorted(theta)
    c = get_connivent(theta, preferences)
    print("Theta = ", theta, end = "\t")
    print("c = ", c, end = "\t")
    print("Banned = ", banned)
    
    if c == None:
        kernels = [theta]
        #kernels += get_kernels(prf, theta)
        for theta in kernels:
            if theta in theta_mins:
                continue
            if len(theta_mins) == 0:
                theta_mins.append(theta)
                return additivity(theta_mins[0]), len(theta_mins[0])
            if theta_better_opt(theta, theta_mins[0]) == 1:
                theta_mins.clear()
                theta_mins.append(theta)
            elif theta_better_opt(theta, theta_mins[0]) == 0:
                theta_mins.append(theta)
        return
    
    its = get_candidate_iterator(c)
    b = list(banned)
    for k in its:
        for candidates in its[k]:
            for can in candidates:
                if len(theta_mins) > 0 and k > additivity(theta_mins[0]):
                    #print("Cutting over", k, " in:", theta , end = " ")
                    break
                if can in theta or can in b:
                    continue
                n_theta = theta + [can]
                if len(theta_mins) > 0 and theta_better_opt(n_theta, theta_mins[0]) == -1:
                    #print("Cutting", c , " in:", theta, end = " ")
                    continue
                print(f"In {theta} Trying candidate {c}")
                dfs_thetas_opt(preferences, n_theta, theta_mins, banned = b)
                if bann_opt:
                    b.append(can)
    return True



def get_min_thetas(preferences, initial_theta = None, bann_opt = True):
    representant = None
    theta_mins = []
    stats = []
    if not initial_theta:
        initial_theta = [EMPTY_SET]
    dfs_thetas_r(preferences, initial_theta, theta_mins = theta_mins, bann_opt = bann_opt)
    return theta_mins, stats

def get_all_thetas(preferences, initial_theta = None, bann_opt = True):
    representant = None
    theta_mins = []
    stats = []
    if not initial_theta:
        initial_theta = [EMPTY_SET]
    dfs_thetas_full(preferences, initial_theta, theta_mins = theta_mins, bann_opt = bann_opt)
    return theta_mins, stats

def get_opt_thetas(preferences, initial_theta = None, bann_opt = True):
    representant = None
    theta_mins = []
    stats = []
    if not initial_theta:
        initial_theta = [EMPTY_SET]
    dfs_thetas_opt(preferences, initial_theta, theta_mins = theta_mins, bann_opt = bann_opt)
    return theta_mins, stats

In [12]:
n_items = 6
items = np.arange(n_items)

prf = sample_preferences_from_order(items, 80, indifference_rate= 0)

In [13]:
t = get_min_thetas(prf, [EMPTY_SET])

=====
Theta =  []	c =  [((), (1,))]	Banned =  []
=====
Theta =  [(1,)]	c =  [((2,), ())]	Banned =  []
=====
Theta =  [(1,), (2,)]	c =  [((0,), (2,)), ((2,), ())]	Banned =  []
=====
Theta =  [(0,), (1,), (2,)]	c =  [((3,), (0,)), ((0,), (2,)), ((2,), ())]	Banned =  []
=====
Theta =  [(0,), (1,), (2,), (3,)]	c =  [((0,), (2,)), ((2,), ()), ((), (0, 2, 4))]	Banned =  []
=====
Theta =  [(0,), (1,), (2,), (3,), (4,)]	c =  [((2, 3, 4), (0,)), ((0,), (2,)), ((2,), ()), ((0, 2, 4, 5), (0, 2, 3, 4, 5)), ((), (0, 2, 4))]	Banned =  []
=====
Theta =  [(0,), (1,), (2,), (3,), (4,), (5,)]	c =  [((2, 3, 4), (0,)), ((0,), (2,)), ((2,), ()), ((0, 2, 4, 5), (0, 2, 3, 4, 5)), ((), (0, 2, 4))]	Banned =  []
=====
Theta =  [(0,), (1,), (2,), (2, 3), (3,), (4,), (5,)]	c =  [((2, 3, 4), (0,)), ((0,), (2,)), ((2,), ()), ((0, 2, 4, 5), (0, 2, 3, 4, 5)), ((), (0, 2, 4))]	Banned =  []
=====
Theta =  [(0,), (0, 2), (1,), (2,), (2, 3), (3,), (4,), (5,)]	c =  [((2, 3), (3,)), ((3,), (0,)), ((0,), (2,)), ((0, 2, 4, 5

In [6]:
t

([[(0,), (1,), (1, 2), (2,), (2, 4), (3,)],
  [(1,), (1, 2), (1, 4), (2,), (3,), (4,)],
  [(1,), (1, 2), (2,), (2, 4), (3,), (4,)],
  [(1,), (1, 2), (2,), (3,), (4,), (4, 5)],
  [(0, 4), (1,), (1, 2), (2,), (2, 4), (3,)],
  [(1, 2), (2,), (3,), (4,), (4, 5), (5,)],
  [(1, 4), (2,), (2, 5), (3,), (4,), (5,)],
  [(1, 2), (1, 5), (2,), (3,), (4,), (4, 5)]],
 [])

In [7]:
#L = get_kernels(prf,union(t[0]))
#for s in L:
#    print(sorted(s))

In [10]:
t = get_min_thetas(prf, [EMPTY_SET])

=====
Theta =  []	c =  [((), (3,))]	Banned =  []
=====
Theta =  [(3,)]	c =  [((), (3,)), ((3,), (2,))]	Banned =  []
=====
Theta =  [(2,), (3,)]	c =  [((), (3,)), ((3,), (2,)), ((1, 2, 5), (1, 5))]	Banned =  []
=====
Theta =  [(1,), (2,), (3,)]	c =  [((), (3,)), ((3,), (2,)), ((1, 2, 5), (1, 5))]	Banned =  []
=====
Theta =  [(1,), (2,), (3,), (5,)]	c =  [((), (3,)), ((3,), (2,)), ((1, 2, 5), (1, 5))]	Banned =  []
=====
Theta =  [(1,), (1, 2), (2,), (3,), (5,)]	c =  [((0, 4), ())]	Banned =  []
=====
Theta =  [(0,), (1,), (1, 2), (2,), (3,), (5,)]	c =  [((1, 2, 3, 4), ()), ((), (3,)), ((3,), (1, 5)), ((2, 3, 4, 5), (1, 2, 5)), ((1, 2, 5), (1, 5)), ((1, 5), (2,))]	Banned =  []
=====
Theta =  [(0,), (1,), (1, 2), (2,), (3,), (4,), (5,)]	c =  [((1, 2, 4, 5), (0, 4)), ((3,), (1, 2, 5)), ((0, 2, 4, 5), (2, 3, 4, 5))]	Banned =  []
=====
Theta =  [(0,), (1,), (1, 2), (2,), (2, 3), (3,), (4,), (5,)]	c =  None	Banned =  []
=====
Theta =  [(0,), (1,), (1, 2), (2,), (2, 4), (3,), (4,), (5,)]	c =  No

In [11]:
t

([[(0,), (1,), (1, 2), (2,), (2, 4), (3,)],
  [(1,), (1, 2), (1, 4), (2,), (3,), (4,)],
  [(1,), (1, 2), (2,), (2, 4), (3,), (4,)],
  [(1,), (1, 2), (2,), (3,), (4,), (4, 5)],
  [(0, 4), (1,), (1, 2), (2,), (2, 4), (3,)],
  [(1, 2), (2,), (3,), (4,), (4, 5), (5,)],
  [(1, 4), (2,), (2, 5), (3,), (4,), (5,)],
  [(1, 2), (1, 5), (2,), (3,), (4,), (4, 5)]],
 [])